# RNN 텍스트 분류기

In [29]:
corpus = [
    '자연어 처리는 재미있다',
    'Python이 자연어 처리보다 쉽다',
    '자연어 처리 공부는 어렵다',
    'Python 활용법을 더 즐겁게 찾아보자'
]

labels = [1, 0, 0, 1]

In [30]:
tokenized_corpus = [sentence.split() for sentence in corpus]

vocab = {}
for tokens in tokenized_corpus:
    for token in tokens:
        if token not in vocab:
            vocab[token] = len(vocab) + 1

indexed_corpus = []
for tokens in tokenized_corpus:
    indexed_sent = [vocab[token] for token in tokens]
    indexed_corpus.append(indexed_sent)
    
max_seq_len = max(len(seq) for seq in indexed_corpus)
def pad_sequences(seq, max_len):
    if len(seq) < max_len:
        seq = seq + [0] * (max_len - len(seq))
    return seq

padded_corpus = [pad_sequences(seq, max_seq_len) for seq in indexed_corpus]

In [31]:
import torch

inputs = torch.tensor(padded_corpus, dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)


In [32]:
# RNN 기반 텍스트 분류기 모델 정의
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super(RNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        _, hiddden = self.rnn(x)
        return self.sigmoid(self.fc(hiddden[-1]))

In [33]:
VOCAB_SIZE = len(vocab) + 1
EMBED_SIZE = 128
HIDDEN_SIZE = 64
NUM_CLASSES = 1

model = RNNClassifier(
    vocab_size=VOCAB_SIZE,
    embed_size=EMBED_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_classes=NUM_CLASSES
)

print(model)

RNNClassifier(
  (embedding): Embedding(15, 128)
  (rnn): RNN(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [34]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 20

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(inputs)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch {epoch + 1}/{epochs} | Loss: {loss.item():.4f}')

Epoch 1/20 | Loss: 0.6731
Epoch 2/20 | Loss: 0.3080
Epoch 3/20 | Loss: 0.1734
Epoch 4/20 | Loss: 0.0869
Epoch 5/20 | Loss: 0.0464
Epoch 6/20 | Loss: 0.0292
Epoch 7/20 | Loss: 0.0197
Epoch 8/20 | Loss: 0.0138
Epoch 9/20 | Loss: 0.0101
Epoch 10/20 | Loss: 0.0076
Epoch 11/20 | Loss: 0.0058
Epoch 12/20 | Loss: 0.0045
Epoch 13/20 | Loss: 0.0036
Epoch 14/20 | Loss: 0.0029
Epoch 15/20 | Loss: 0.0024
Epoch 16/20 | Loss: 0.0020
Epoch 17/20 | Loss: 0.0017
Epoch 18/20 | Loss: 0.0015
Epoch 19/20 | Loss: 0.0013
Epoch 20/20 | Loss: 0.0011


In [35]:
test_texts = [
    '자연어 처리는 재미있다',
    'Pyhton은 어렵다'
]

def preprocess_sentence(sentence, vocab, max_len):
    tokens = sentence.split()
    indices = [vocab.get(token, 0) for token in tokens]
    indices = pad_sequences(indices, max_len)
    return torch.tensor(indices, dtype=torch.long)

test_inputs = []
for sent in test_texts:
    test_inputs.append(preprocess_sentence(sent, vocab, max_seq_len))
    
test_inputs = torch.stack(test_inputs)

In [36]:
model.eval()
with torch.no_grad():
    outputs = model(test_inputs)
    print(outputs)

tensor([[0.9983],
        [0.9954]])
